<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js076_airFlowsLB2D9V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js076_airFlowsLB2D9V / def exec_html_js() ... exec me first

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js076_airFlowsLB2D9V</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js076_airFlowsLB2D9V
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.12.22 created, last updated on 2018.11.29
//    ver 0.0.1  2019.01.22 v1, last updated on 2021.08.04
//    ver 0.0.2  2021.11.04 v2, last updated on 2021.11.04
//    ver 0.0.3  2023.04.19 v3, last updated on 2023.09.03
//
// -------------------- Lattice Boltzmann Method - 2D9V model  --- fluid dynamics
//
// - method: Lattice Boltzmann Method
//    2D9V model - velocity distribution in lattice i,j:
//       base vectors: e[0]=[ 0, 0], e[1]=[ 1, 0], e[2]=[ 1,-1], e[3]=[ 0,-1], e[4]=[-1,-1],
//                     e[5]=[-1, 0], e[6]=[-1, 1], e[7]=[ 0, 1], e[8]=[ 1, 1]
//       velocity distribution at lattice i,j: fv[i][j][k]*e[k], k=0,1,2,...,8
//
//      (1) set lattice:
//          density: rho[i][j],
//          flow velocity: ux[i][j],uy[i][j]
//          velosity distribution: f[i][j][k] (k=0,1,2,...,8)
//
//      (2) streaming  f(r+ek*dt,t+dt) = f(r,t)
//         ftx[i][j][0] <-- ftp[i][j][0]     // e[0]=[ 0, 0]
//         ftx[i][j][1] <-- ftp[i-1][j][1]   // e[1]=[ 1, 0]
//         ftx[i][j][2] <-- ftp[i-1][j+1][2] // e[2]=[ 1,-1]
//         ftx[i][j][3] <-- ftp[i][j+1][3]   // e[3]=[ 0,-1]
//         ftx[i][j][4] <-- ftp[i+1][j+1][4] // e[4]=[-1,-1]
//         ftx[i][j][5] <-- ftp[i+1][j][5]   // e[5]=[-1, 0]
//         ftx[i][j][6] <-- ftp[i+1][j-1][6] // e[6]=[-1, 1]
//         ftx[i][j][7] <-- ftp[i][j-1][7]   // e[7]=[ 0, 1]
//         ftx[i][j][8] <-- ftp[i-1][j-1][8] // e[8]=[ 1, 1]
//
//      (3) scattering  f(r,t+dt) = f(r,t+dt) - omega*(f(r,t+dt)-feq))
//         rho = sigma(f[i][j][k], k=0,1,2,...,8)
//         vx = sigma(e[k][0]*f[i][j][k], k=0,1,2,...,8)/rho
//         vy = sigma(e[k][1]*f[i][j][k], k=0,1,2,...,8)/rho
//         feq[k]: equilibrium velocity distribution (rho*u = sum(feq[k]*e[k],k=0,...8)
//                 feq[k] = w[k]*rho(r)*(1+3*(e[k].u)+4.5*(e[k].u)^2-1.5u^2) (D2Q9 at c=dx/dt=1)
//         f[i][j][k] = f[i][j][k] - omega*(f[i][j][k]-feq[k])
//           omega = 1/tau = 1/(3*kineticViscosity+0.5),(2D,dx=1,dt=1). tau:relaxation time
//
//         at boundary
//           wall: bounce-back ( E <--> W, N <--> S, NE <--> SW, NW <--> SE )
//           velocity boundary: set rho,ux,uy and feq
//
//      goto (2)
//
// --------------------
*/

const airFlowsLB2D9V = (function(){ // ====================  airFlowsLB2D9V Module  ====================

	const g_nxMax = 360;			// maximum array x-size of field
	const g_nyMax = 360;			// maximum array y-size of field

	let g_sysTime = 0.0;			// system time
	let g_dt = 1.0;					// time step
	let g_dx = 1.0;					// x-division
	let g_dy = 1.0;					// y-division
	let g_tpp = 0;					// present time v9[tpp][i][j], tpp = 0 or 1
	let g_tnx = 1;					// next    time v9[tnx][i][j], tnx = 1 or 0
	let g_status = 'STABLE';		// system status (in string)

	let g_themeNum = 0;				// theme mnumber 0 - 6
	let g_themeStr = "theme";		// theme eg. "gas expansion"
	let g_NNx = 180;				// array x-size of field
	let g_NNy = 180;				// array y-size of field
	let g_drawScale = 2;			// draw scale
	let g_vxBoundary = 0.1;			// periodic condition : vx=0.1 (sound speed = 1.0) at x=0
	let g_vyBoundary = 0.0;			// periodic condition : vy=0.0 (sound speed = 1.0) at x=0

	let g_xProbe = 360/2;			// probe position x
	let g_yProbe = 180/2;			// probe position y

	//              0    1    2    3    4    5    6    7    8
	const g_feq = [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0 ]; // equilibrium velocity distribution
	const g_kind = dimInt2( g_nxMax, g_nyMax );		// kind[ix][iy] 0:free space,1:wall,2:velocity boundary
	const g_v9 = dim4( 2, g_nxMax, g_nyMax, 9 );	// velocity distribution v9[tpp/tnx][ix][iy][0-8]
	const g_rho = dim2( g_nxMax, g_nyMax );			// density rho[ix][iy]
	const g_ux = dim2( g_nxMax, g_nyMax );			// x-component of flow velocity (sound speed = 1.0)
	const g_uy = dim2( g_nxMax, g_nyMax );			// y-component of flow velocity (sound speed = 1.0)


	function dim2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Float64Array( nj );
		}
		return a;
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dim4( ni, nj, nk, nl ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = [];
				for (let k=0; k<nk; k++) {
					a[i][j][k] = new Float64Array( nl );
				}
			}
		}
		return a;
	}


	// --------------------  set theme and user parameter  --------------------

	const g_themeParam = [
		// 0:themeStr            1:NNx 2:NNy 3:sc 4:nue 5:vxFlow 6:rhoMag
		["gas expansion",        180,  180,  2,   0.04, 0.000,  1.0  ],
		["gas expansion+object", 180,  180,  2,   0.06, 0.000,  1.0  ],
		["vortex",               180,  180,  2,   0.06, 0.000,  3.0  ],
		["Couette flow",         180,   90,  2,   0.20, 0.200, 10.0  ],
		["cavity flow",          120,  120,  3,   0.10, 0.200, 30.0  ],
		["wind tunnel(180x90)",  180,   90,  2,   0.02, 0.100, 30.0  ],
		["wind tunnel(120x60)",  120,   60,  3,   0.02, 0.100, 30.0  ]
	];

	const g_userParam = {
		powderON : true,
		probeON : false,
		nue : 0.06,
		vxFlow : 0.10,
		rhoMag : 1.0
	};

	function setThemeAndUserParam(theme) {
		g_themeNum = theme;
		g_themeStr = g_themeParam[theme][0];
		g_NNx = g_themeParam[theme][1];
		g_NNy = g_themeParam[theme][2];
		g_drawScale = g_themeParam[theme][3];
		g_userParam.nue = g_themeParam[theme][4];
		g_userParam.vxFlow = g_themeParam[theme][5];
		g_userParam.rhoMag = g_themeParam[theme][6];
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme ) {
		g_status = 'STABLE';
		g_sysTime = 0.0;
		setThemeAndUserParam(theme);
		g_vxBoundary = g_userParam.vxFlow;
		setInitialLattice(theme);
		powder.setInitialPosition(); // powder
		g_xProbe = 360/2; g_yProbe = 180/2;
	}

	function setInitialLattice(theme) {
		const nnx=g_NNx, nny=g_NNy;
		let rh = 1.0, vx = 0.0, vy = 0.0;
		// free space
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				g_kind[i][j] = 0;
				if (theme==0 || theme==1) { // 0:gas expansion 1:gas expansion+object
					const x0 = nnx/3;
					const y0 = nny/3;
					rh = ((i-x0)*(i-x0)+(j-x0)*(j-x0)<30*30) ? 1.5 : 0.8;
				} else if (theme==2) { // 2:vortex
					const x0 = nnx/2;
					const y0 = nny/2;
					const r = 0.4*nnx;
					if ( (i-x0)*(i-x0)+(j-y0)*(j-y0)<r*r) {
						vx = -0.4*(j-y0)/r;
						vy = 0.4*(i-x0)/r;
					} else {
						vx = 0.0;
						vy = 0.0;
					}
				} else if (theme==3) { // 3:Couette flow
					vx = g_userParam.vxFlow;
					vy = 0.0;
					rh = 1.0;
				} else if (theme==5 || theme==6) { // 5-6:wind tunnel
					vx = g_userParam.vxFlow;
					vy = 0.0;
					rh = 1.0;
				}
				setEquiLattice(g_v9[g_tpp][i][j], rh, vx, vy, i, j);
			}
		}
		// set boundary
		if (theme==1) { // gas expansion+object
			for (let i=80; i<120; i++) {
				for (let j=70; j<130; j++) {
					if ((i-100)*(i-100)/(10*10)+(j-100)*(j-100)/(20*20)<1) {
						setWall(i,j);
					}
				}
			}
		} else if (theme==3) { // 3:Couette flow
			for (let i=0; i<nnx; i++) {
				setWall(i,0);
				setWall(i,nny-1);
			}
			for (let j=1; j<nny-1; j++) {
				g_kind[0][j] = 2; // velocity boundary
			}
		} else if (theme==4) { // 4:cavity flow
			for (let i=0; i<nnx; i++) {
				g_kind[i][0] = 2; // velocity boundary
				setWall(i,nny-1);
			}
			for (let j=1; j<nny-1; j++) {
				setWall(0,j);
				setWall(nnx-1,j);
			}
		} else if (theme==5 || theme==6) { // 5-6:wind tunnel
			const r = Math.floor(nny/10);
			const i0 = Math.floor(nnx/5);
			for (let j=Math.floor(nny/2)-r; j<Math.floor(nny/2)+r; j++) {
				setWall(i0,j);
				setWall(i0+1,j);
			}
			for (let j=0; j<nny; j++) {
				g_kind[0][j] = 2; // velocity boundary
			}
			for (let i=1; i<nnx; i++) {
				g_kind[i][0] = 2; // velocity boundary
			}
		}
	}

	function setWall(i,j) {
		g_kind[i][j] = 1; // wall
		for (let k=0; k<9; k++) {
			g_v9[g_tpp][i][j][k] = 0.0;
		}
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( nCalc ) {
		if (g_status=='STABLE') {
			for (let i=0; i<nCalc; i++) {
				streaming();
				scattering(g_userParam.nue); //nue: kinetic viscosity
				if (g_userParam.powderON) powder.move(); // powder
				if (g_status != 'STABLE') break;
				g_sysTime += g_dt;
				g_tpp = g_tnx;
				g_tnx = (g_tnx+1)%2;
			}
		}
		return g_status;
	}

	// --- (2) streaming

	function streaming() {
		const nnx=g_NNx, nny=g_NNy;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const ip = (i+1)%nnx;
				const im = (i-1+nnx)%nnx;
				const jp = (j+1)%nny;
				const jm = (j-1+nny)%nny;
				const fij = g_v9[g_tnx][i][j];
				fij[0] = g_v9[g_tpp][i][j][0];   // e[0]=[ 0, 0]
				fij[1] = g_v9[g_tpp][im][j][1];  // e[1]=[ 1, 0]
				fij[2] = g_v9[g_tpp][im][jp][2]; // e[2]=[ 1,-1]
				fij[3] = g_v9[g_tpp][i][jp][3];  // e[3]=[ 0,-1]
				fij[4] = g_v9[g_tpp][ip][jp][4]; // e[4]=[-1,-1]
				fij[5] = g_v9[g_tpp][ip][j][5];  // e[5]=[-1, 0]
				fij[6] = g_v9[g_tpp][ip][jm][6]; // e[6]=[-1, 1]
				fij[7] = g_v9[g_tpp][i][jm][7];  // e[7]=[ 0, 1]
				fij[8] = g_v9[g_tpp][im][jm][8]; // e[8]=[ 1, 1]
			}
		}
	}

	// --- (3) scattering

	function scattering(kineticViscosity) {
		const nnx=g_NNx, nny=g_NNy;
		const omega = 1.0/(3.0*kineticViscosity+0.5); // omega = 1/tau; tau: relaxation time
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const fij = g_v9[g_tnx][i][j];
				if (g_kind[i][j]==0) { // free space
					const rh = fij[0]+fij[1]+fij[2]+fij[3]+fij[4]+fij[5]+fij[6]+fij[7]+fij[8]; // rho;
					if (rh>=0) {
						const vx = (fij[1]+fij[2]-fij[4]-fij[5]-fij[6]+fij[8])/rh; // flow vx;
						const vy = (-fij[2]-fij[3]-fij[4]+fij[6]+fij[7]+fij[8])/rh; // flow vy;
						setEquiLattice(g_feq, rh, vx, vy, i, j); // set equilibrium feq at rho,vx,vy
						for (let k=0; k<9; k++) {
							fij[k] -= omega*(fij[k]-g_feq[k]);
						}
					} else {
						g_status = "UNSTABLE at i="+i+",j="+j+",g_rho="+rh.toFixed(3)+"<br>"+
									"press reset or another theme";
					}
				} else if (g_kind[i][j]==1) { // wall: bounce-back boundary
					let m;
					fij[0] = 0.0;
					m = fij[1]; fij[1] = fij[5]; fij[5] = m; // e[1]=[ 1, 0], e[5]=[-1, 0]
					m = fij[2]; fij[2] = fij[6]; fij[6] = m; // e[2]=[ 1,-1], e[6]=[-1, 1]
					m = fij[3]; fij[3] = fij[7]; fij[7] = m; // e[3]=[ 0,-1], e[7]=[ 0, 1]
					m = fij[4]; fij[4] = fij[8]; fij[8] = m; // e[4]=[-1,-1], e[8]=[ 1, 1]
				} else if (g_kind[i][j]==2) { // velocity boundary
					setEquiLattice(fij, 1.0, g_vxBoundary, g_vyBoundary, i, j);
				}
			}
		}
	}

	function setEquiLattice(veq, rh, vx, vy, i, j) {
		g_rho[i][j] = rh;
		g_ux[i][j] = vx;
		g_uy[i][j] = vy;
		setEquilibriumV9(veq, rh, vx, vy);
	}

	function setEquilibriumV9(veq, rh, vx, vy) {
		const w0rh = rh*4.0/9.0, w1rh = rh/9.0, w2rh = rh/36.0;
		const c0 = 1.0 - 1.5*(vx*vx+vy*vy);
		const c1v = vx;     // e[1]=[ 1, 0], (e.u) = (1,0).(vx,vy) = vx
		const c2v = vx-vy;  // e[2]=[ 1,-1]
		const c3v = -vy;    // e[3]=[ 0,-1]
		const c4v = -vx-vy; // e[4]=[-1,-1]
		const c5v = -vx;    // e[5]=[-1, 0]
		const c6v = -vx+vy; // e[6]=[-1, 1]
		const c7v = vy;     // e[7]=[ 0, 1]
		const c8v = vx+vy;  // e[8]=[ 1, 1]
		// feq[k] = w[k]*rho(r)*(1+3*(e[k].u)+4.5*(e[k].u)^2-1.5u^2)
		veq[0] = w0rh*c0;
		veq[1] = w1rh*(c0 + 3.0*c1v + 4.5*c1v*c1v);
		veq[2] = w2rh*(c0 + 3.0*c2v + 4.5*c2v*c2v);
		veq[3] = w1rh*(c0 + 3.0*c3v + 4.5*c3v*c3v);
		veq[4] = w2rh*(c0 + 3.0*c4v + 4.5*c4v*c4v);
		veq[5] = w1rh*(c0 + 3.0*c5v + 4.5*c5v*c5v);
		veq[6] = w2rh*(c0 + 3.0*c6v + 4.5*c6v*c6v);
		veq[7] = w1rh*(c0 + 3.0*c7v + 4.5*c7v*c7v);
		veq[8] = w2rh*(c0 + 3.0*c8v + 4.5*c8v*c8v);
	}

	// -------------------- utility

	function vorticity(i, j) {
		if (i==0 || i==g_NNx-1 || j==0 || j==g_NNy-1) return 0.0;
		return (g_uy[i+1][j]-g_uy[i-1][j])/(2*g_dx)-(g_ux[i][j+1]-g_ux[i][j-1])/(2*g_dy);
	}

	function speed(i,j) {
		const uxij= g_ux[i][j], uyij= g_uy[i][j];
		return Math.sqrt( uxij*uxij+uyij*uyij );
	}

	function totalRho() {
		const nnx=g_NNx,nny=g_NNy;
		let s=0.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				if (g_kind[i][j]==0) s += g_rho[i][j];
			}
		}
		return s;
	}

	function meanRho() {
		const nnx=g_NNx, nny=g_NNy;
		let n=0, s=0.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				if (g_kind[i][j]==0) {
					n++;
					s += g_rho[i][j];
				}
			}
		}
		return s/n;
	}


	// --------------------  powder (LB2D)  --------------------

	// external: g_NNx,g_NNy,g_dx,g_dy,g_dt,g_ux[][],g_uy[][],g_kind[][]

	const powder = {};   // namespace of powder module
	powder.n = 300;    // n: number of powder
	powder.x = [];     // x-position of i-th powder
	powder.y = [];     // y-position of i-th powder

	powder.setInitialPosition = function() {
		const nn=powder.n, px=powder.x, py=powder.y;
		for (let i=0; i<nn; i++) {
			px[i] = (g_NNx-2)*Math.random()+1;
			py[i] = (g_NNy-2)*Math.random()+1;
		}
	};

	powder.move = function() {
		const nnx=g_NNx, nny=g_NNy, nn=powder.n, ta=g_dt*(1.0/g_dx), px=powder.x, py=powder.y;

		for (let i=0; i<nn; i++) {
			const ix = Math.floor(px[i]);
			const iy = Math.floor(py[i]);
			px[i] += g_ux[ix][iy]*ta;
			py[i] += g_uy[ix][iy]*ta;
			if (px[i]>=nnx) px[i] -= nnx;
			if (px[i]<0) px[i] += nnx;
			if (py[i]>=nny) py[i] -= nny;
			if (py[i]<0) py[i] += nny;
			if (g_kind[ix][iy]==1) {
				px[i] = (nny-2)*Math.random()+1;
				py[i] = (nny-2)*Math.random()+1;
			} else if (g_kind[ix][iy]==2) {
				if ( Math.random()<0.001 ) {
					px[i] = (nny-2)*Math.random()+1;
					py[i] = (nny-2)*Math.random()+1;
				}
			}
		}
	};


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme )
		evolve:			timeEvolution,			// timeEvolution( nCalc )

		setPowderFlag:	function( sw ) { g_userParam.powderON = sw; },
		setProbeFlag:	function( sw ) { g_userParam.probeON = sw; },

		getSysParam:	function() { return [ g_NNx, g_NNy, g_dx, g_dt, powder.n,  ]; },
		getParam:		function() { return [ g_drawScale, g_userParam.nue, g_userParam.vxFlow, g_userParam.rhoMag ]; },
		getNow:			function() { return [ g_sysTime, g_status, g_themeStr, meanRho() ]; },
		getCellKind:	function(i,j) { return g_kind[i][j]; }, // kind[ix][iy] 0:free space,1:wall,2:velocity boundary
		getFlow:		function(i,j) { return [ g_ux[i][j], g_uy[i][j] ]; },
		getDensity:		function(i,j) { return g_rho[i][j]; },
		getVorticity:	vorticity,				// vorticity(i,j)
		getSpeed:		speed,					// speed(i,j)
		getPowderPos:	function(ipow) { return [ powder.x[ipow], powder.y[ipow] ]; },
	};

})(); // ====================  airFlowsLB2D9V end  ====================


const js076 = (function(){ // ====================  js Module  ====================

	const theModule = airFlowsLB2D9V;
	const xCanvasSize = 500;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;					// = canvas.getContext('2d');

	let v_theme = 6; // 6:wind tunnel(120x60)
	let v_nCalc = 6; // 6 (calc/frame)

	let p_NNx, p_NNy, p_dx, p_dt, p_nPowder; // <-- theModule.getSysParam()
	let p_drawScale, p_nue, p_vxFlow, p_rhoMag; // <-- theModule.getParam()
	let sysTime, status, themeStr, meanRho;
  let cellKindArray = [];
  let fieldArray = [];
  let vectorArray = [];


	let xBoxSize = 450;
	let dispMode = 7; // 7: grid2D density + powder
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	//let inStepFlag = false;

  let breakFlag = false;
  let getFieldFlag = true;
  let fieldKind = 1 // cellKind, density, flow

	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();
		viewHome();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme ); // ( nn, BoxSizeInNM, contTemp )
			imgField = null;
			[ p_NNx, p_NNy, p_dx, p_dt, p_nPowder ] = theModule.getSysParam();
			[ p_drawScale, p_nue, p_vxFlow, p_rhoMag ] = theModule.getParam();
			xBoxSize = ( v_theme==3 || v_theme==5 || v_theme==6 ) ? 450 : 320;
			// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
			g3d.init( p_NNx, p_NNy, p_NNy/2, p_dx, xCanvasSize, yCanvasSize, xBoxSize, 20 );
			g3d.drawWallGrid2D.threshold = 0.0;
			g3d.drawWallVectorField2D.threshold = 0.0;
      let getFieldFlag = true;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_nCalc );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_nCalc );
			//inStepFlag = true;
		}

		draw( ctx, dispMode );

    if ( getFieldFlag ) setFieldData( fieldKind );

		requestAnimationFrame(animate);
	}

  function setFieldData( fieldKind ) {
    if (fieldKind==1) { // fieldKind: 1: cellKind, density, flow
      for (let i=0; i<p_NNx; i++) {
        cellKindArray[i] = [];
        fieldArray[i] = [];
        vectorArray[i] = [];
        for (let j=0; j<p_NNy; j++) {
          cellKindArray[i][j] = theModule.getCellKind(i,j);
          fieldArray[i][j] = theModule.getDensity(i,j);
          vectorArray[i][j] = theModule.getFlow(i,j);
        }
      }
    }
  }


	// --------------------  draw  --------------------

	function draw( ctx, dispMode ) {

		const densMag = 200;
		const densityFunc = function(i,j) { return densMag*(theModule.getDensity(i,j)-1.0); };
		const wallFunc = function(i,j) { return (theModule.getCellKind(i,j)==1); };
		const vorticityColorFunc = function(i,j) {
			const hue = Math.floor(36120.0 - 5000.0*theModule.getVorticity(i,j)) % 360;
			return `hsl(${hue},100%,50%)`;
		}

		const xp = 40, yp = 40, sc = p_drawScale;

		// clear
		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		if ( imgField == null ) {
			const xSize = p_NNx*g_imgScale, ySize = p_NNy*g_imgScale;
			imgField = ctx.getImageData( xp, yp, xSize, ySize );
		}

		if (dispMode==0) {
			dispText( "density(x,y) + flow" );
			drawImgField( ctx, 0, 1, xp, yp, sc );
		} else if (dispMode==1) {
			dispText( "density(x,y) + powder" );
			drawImgField( ctx, 0, 2, xp, yp, sc );
		} else if (dispMode==2) { // vorticity + powder
			dispText( "vorticity(x,y) + powder" );
			drawImgField( ctx, 1, 2, xp, yp, sc );
		} else if (dispMode==3) { // speed + flow
			dispText( "(speed(x,y)-vx0) + powder" );
			drawImgField( ctx, 2, 2, xp, yp, sc );

		} else if (dispMode==4) {
			dispText( "grid: density(x,y)" );
			// g3d.drawWallGrid2D( ctx, rotAngle, wallFunc, zFunc, colorFactor, inc [, showBox] )
			g3d.drawWallGrid2D( ctx, 0.0, wallFunc, densityFunc, 0.5, 2 );

		} else if (dispMode==5) {
			dispText( "grid: vorticity(x,y)" );
			// g3d.drawWallGrid2D( ctx, rotAngle, wallFunc, zFunc, colorFactor, inc [, showBox] )
			g3d.drawWallGrid2D( ctx, 0.0, wallFunc, densityFunc, vorticityColorFunc, 2 );

		} else if (dispMode==6) {
			dispText( "vector grid: flow(x,y) + z:density" );
			const vFunc = function(i,j) {
				let vx, vy, mag=50.0;
				[ vx, vy ] = theModule.getFlow(i,j);
				return [ vx*mag, vy*mag ];
			}
			// g3d.drawWallVectorField2D(ctx, rotAngle, wallFunc, zFunc, vFunc, colorMode, inc [, showBox] )
			g3d.drawWallVectorField2D(ctx, 0.0, wallFunc, densityFunc, vFunc, 0, 2 );

		} else if (dispMode==7) {
			dispText( "grid: density(x,y) + powder" );
			// g3d.drawWallGrid2D( ctx, rotAngle, wallFunc, zFunc, colorFactor, inc [, showBox] )
			g3d.drawWallGrid2D( ctx, 0.0, wallFunc, densityFunc, 0.5, 2 );
			//drawPowder3D(ctx, nPowder, rmag );
			drawPowder3D(ctx, p_nPowder, densMag );
			let scale, xPos, yPos; [ scale, xPos, yPos ] = g3d.scxpypFunc();
			g3d.plotNearEdge(ctx, scale, xPos, yPos, "#999900" ); // override near edge of the box

		} else if (dispMode==8) {
			dispText( "grid: z:density(x,y), color:vorticity(x,y) + powder" );
			// g3d.drawWallGrid2D( ctx, rotAngle, wallFunc, zFunc, colorFactor, inc [, showBox] )
			g3d.drawWallGrid2D( ctx, 0.0, wallFunc, densityFunc, vorticityColorFunc, 2 );
			//drawPowder3D(ctx, nPowder, rmag );
			drawPowder3D(ctx, p_nPowder, densMag );
			let scale, xPos, yPos; [ scale, xPos, yPos ] = g3d.scxpypFunc();
			g3d.plotNearEdge(ctx, scale, xPos, yPos, "#999900" ); // override near edge of the box
		}

		// caption
		[ sysTime, status, themeStr, meanRho ] = theModule.getNow();
		const xBox = p_NNx*p_dx, yBox = p_NNy*p_dx;
		ctx.fillStyle = "#888888";
		ctx.fillText(`periodic box = ${xBox} x ${yBox},  time = ${sysTime.toFixed(1)}`, 40, yCanvasSize-30);
		ctx.fillText(`nue = ${p_nue.toFixed(2)},  vx0 = ${p_vxFlow.toFixed(3)} x c(sound speed)`, 40, yCanvasSize-10);
		// Re = (vxFlow/nue)*objectLength

		let msg = `the system is ${status}.`;
		if ( status!='STABLE' ) {
			msg = msg+"<br> !!! please select lower Re/L, and press reset button.";
		}
		document.getElementById("text_caption").innerHTML =	msg;

		function dispText( str ) {
			ctx.fillStyle = "#888888";
			ctx.fillText( str, 40, yCanvasSize-50 );
		}
	}


	// --- draw image field

	function drawImgField( ctx, FieldMode, addon, xp, yp, sc ) {
		const nnx=p_NNx, nny=p_NNy;

		ctx.strokeStyle = "rgb(80, 80, 80)"; // box
		ctx.strokeRect(xp-1,yp-1,p_NNx*sc+2,p_NNy*sc+2);

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				if (theModule.getCellKind(i,j)==1) continue;
				let hue;
				if (FieldMode==0) { // density
					hue = Math.floor(36120.0 - 120.0*p_rhoMag*(theModule.getDensity(i,j)-1.0)) % 360;
				} else if (FieldMode==1) { // vorticity
					hue = Math.floor(36120.0 - 5000.0*theModule.getVorticity(i,j)) % 360;
				} else if (FieldMode==2) { // speed
					let vx, vy;
					[ vx, vy ] = theModule.getFlow(i,j);
					hue = Math.floor(36120.0 - 1200.0*(Math.sqrt(vx*vx+vy*vy)-1.0)) % 360;
				}
				//ctx.fillStyle = `hsl(${hue},100%,50%)`;
				//ctx.fillRect(i*sc+xp,(nny-j-1)*sc+yp,sc,sc);
				setSquare(i*sc,(nny-j-1)*sc,sc,hue);
			}
		}
		ctx.putImageData( imgField, xp, yp );
		if (addon==1) drawFlow( ctx, 100.0, xp, yp, sc );
		if (addon==2) drawPowder( ctx, p_nPowder, xp, yp, sc );
	}

	function drawFlow( ctx, mag, xp, yp, sc ) {
		const nnx=p_NNx, nny=p_NNy;

		ctx.lineWidth = 1;
		for (let i=0; i<nnx; i+=4) {
			for (let j=0; j<nny; j+=4) {
				if ( theModule.getCellKind(i,j)==0 ) { // 0:free space, 1:wall
					let vx, vy;
					[ vx, vy ] = theModule.getFlow(i,j);
					const color = (vx>=0) ? "#6666ff" : "#ff4488";
					const x = (i+0.5)*sc, y = (nny-j-0.5)*sc;
					g3d.drawLine( ctx, xp+x, yp+y, xp+x+vx*mag, yp+y-vy*mag, color );
				}
			}
		}
	}

	function drawPowder(ctx, nPowder, xp, yp, sc ) {
		const nny=p_NNy;

		ctx.fillStyle = "#ff80ff";
		for (let i=0; i<nPowder; i++) {
			let x, y;
			[ x, y ] = theModule.getPowderPos(i);
			ctx.fillRect( xp+x*sc, yp+(nny-y)*sc, 2, 2 );
		}
	}


	function drawPowder3D(ctx, nPowder, rmag ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nny=p_NNy, hh = 1.0;

		const r = 2, color = "#ff80ff";
		for (let i=0; i<nPowder; i++) {
			let x, y;
			[ x, y ] = theModule.getPowderPos(i);
			const ii = Math.floor(x), jj = Math.floor(y)
			const xx = x*hh, yy = (nny - y)*hh
			const z = rmag*(theModule.getDensity(ii,jj)-1.0) + g3d.cz0;
			g3d.drawRotatedDisc( ctx, xx, yy, z, r, color, sc,xp,yp)
		}
	}


	// -------------------- image field modue  --------------------
	// ver 0.0.0  2023.04.20

	let imgField = null;		// image in draw()
	const g_imgScale = 3;
	const g_hue = dimInt2(3,361); // red:hue[0][deg]  green:hue[1][deg]  blue:hue[2][deg]
	const g_hueColor = [];
	setHueColor();

	function dimInt2(ni,nj) {
		let a=[];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = 0;
			}
		}
		return a;
	}

	function setHueColor() {
		if (setHueColor.ok==true) return;
		let x,r,g,b;
		for (let deg=0; deg<361; deg++) {
			if (deg<60) {
				x = deg;
				g_hue[0][deg] = 255;
				g_hue[1][deg] = Math.floor(255.0*x/60.0);
				g_hue[2][deg] = 0;
			} else if (deg<120) {
				x = deg-60;
				g_hue[0][deg] = Math.floor(255.0*(60.0-x)/60.0);
				g_hue[1][deg] = 255;
				g_hue[2][deg] = 0;
			} else if (deg<180) {
				x = deg-120;
				g_hue[0][deg] = 0;
				g_hue[1][deg] = 255;
				g_hue[2][deg] = Math.floor(255.0*x/60.0);
			} else if (deg<240) {
				x = deg-180;
				g_hue[0][deg] = 0;
				g_hue[1][deg] = Math.floor(255.0*(60.0-x)/60.0);
				g_hue[2][deg] = 255;
			} else if (deg<360) {
				x = deg-240;
				g_hue[0][deg] = Math.floor(255.0*x/120.0);
				g_hue[1][deg] = 0;
				g_hue[2][deg] = Math.floor(255.0*(120.0-x)/120.0);
			} else {
				g_hue[0][deg] = 0; g_hue[1][deg] = 0; g_hue[2][deg] = 0;
			}
			g_hueColor[deg] = 'rgb('+g_hue[0][deg]+','+g_hue[1][deg]+','+g_hue[2][deg]+')';
		}
		setHueColor.ok = true;
	}
	setHueColor.ok = false;

	function setSquare(ix,iy,aSize,colIndex) {
		const xSize = p_NNx*g_imgScale, data = imgField.data;

		for (let j=iy; j<iy+aSize; j++) {
			for (let i=ix; i<ix+aSize; i++) {
				const idx = (j*xSize+i)*4;
				data[idx] = g_hue[0][colIndex];
				data[idx+1] = g_hue[1][colIndex];
				data[idx+2] = g_hue[2][colIndex];
				data[idx+3] = 255;
			}
		}
	}


	// --------------------  graphics 3D (field) module  --------------------
	//
	// ver 0.0.1  2018.12.16  last updated on 2023.03.01
	// ver 0.0.2  2023.03.03  last updated on 2023.06.01

	let g_NNx, g_NNy, g_NNz, g_dx, g_dy, g_dz, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift;

	const g3d = {};				// namespace of graphic 3D module

	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.xMax = 0.0;				// x-length of box
	g3d.yMax = 0.0;				// y-length of box
	g3d.zMax = 0.0;				// z-length of box
	g3d.cx0 = 0.0;				// x-component of rotate center
	g3d.cy0 = 0.0;				// y-component of rotate center
	g3d.cz0 = 0.0;				// z-component of rotate center
	g3d.Ax = -Math.PI/15.0;		// rotate angle around x-axis
	g3d.Ay = -Math.PI/15.0;		// rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.xApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.yApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.zApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pxApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pyApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pzApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
					[2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid

	// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
	g3d.init = function( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_NNx = NNx; g_NNy = NNy; g_NNz = NNz;
		g_dx = dx; g_dy = dx; g_dz = dx;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.setSize();
	}

	g3d.setSize = function() {
		g3d.xMax = g_NNx*g_dx;		// x-length of box
		g3d.yMax = g_NNy*g_dy;		// y-length of box
		g3d.zMax = g_NNz*g_dz;		// z-length of box
		g3d.cx0 = 0.5*g3d.xMax;		// x-component of rotate center
		g3d.cy0 = 0.5*g3d.yMax;		// y-component of rotate center
		g3d.cz0 = 0.5*g3d.zMax;		// z-component of rotate center
	};

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = g_xCanvasSize/2, yCenter = g_yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_NNx)*g_NNy;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = xBoxSize/(g_NNx*g_dx)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotateXY = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate param
		g3d.rotateApexXY();     // box Apex rotate--> pxApex[i],pyApex[i],pzApex[i]
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g3d.xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g3d.yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g3d.zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g3d.xMax;
		g3d.cy0 = 0.5*g3d.yMax;
		g3d.cz0 = 0.5*g3d.zMax;
	};

	g3d.rotateApexXY = function() { // rotate box apex
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.drawRotatedDisc = function(ctx, x,y,z,r,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1 = cosAy*(x-cx0)+sinAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cx0
		const y1 = cosAx*(y-cy0)-sinAx*(z-cz0) + cy0
		//z1 =-sinAy*(x-cx0)+cosAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cz0
		g3d.drawDisc(ctx, x1*sc+xp,y1*sc+yp,r,color);
	};

	g3d.drawRotatedLine = function(ctx, x1,y1,z1,x2,y2,z2,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1p = cosAy*(x1-cx0)+sinAy*(sinAx*(y1-cy0)+cosAx*(z1-cz0)) + cx0
		const y1p = cosAx*(y1-cy0)-sinAx*(z1-cz0) + cy0
		const x2p = cosAy*(x2-cx0)+sinAy*(sinAx*(y2-cy0)+cosAx*(z2-cz0)) + cx0
		const y2p = cosAx*(y2-cy0)-sinAx*(z2-cz0) + cy0
		g3d.drawLine(ctx, x1p*sc+xp,y1p*sc+yp,x2p*sc+xp,y2p*sc+yp,color);
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1=g3d.pxApex[iApex]*sc+xp, y1=g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2=g3d.pxApex[iApex]*sc+xp, y2=g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function(ctx, x, y, r, color) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	// --------------------  end of graphics 3D (field) module  --------------------

	// g3d_extension grid2d  created 2023.06.17, last updated 2023.06.17
	// g3d.drawWallGrid2D( ctx, rotAngle, wallFunc, zFunc, colorFactor, inc [, showBox] )
	g3d.drawWallGrid2D = function ( ctx, rotAngle, wallFunc, zFunc, colorFactor, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx = g_NNx, nny = g_NNy, threshold = g3d.drawWallGrid2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let jj=0; jj<nny; jj+=inc) {
			let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
			for (let ii=0; ii<nnx; ii++) {
				let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
				if (i<0 || i+1>nnx-1) continue;

				if ( wallFunc(i,nny-j-1) ) {
					const  x = i*g_dx, y = j*g_dx, z = 0, z1 = 2*g3d.cz0;
					g3d.drawRotatedLine(ctx, x,y,z,x,y,z1,'#888888',sc,xp,yp);
					continue;
				}

				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i+1,nny-j-1);
				const x1 = (i+1)*g_dx, y1 = j*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				if (j<0 || j+1>nny-1) continue;

				if ( wallFunc(i,nny-j-1) ) {
					continue;
				}

				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i,nny-j-2);
				const x1 = i*g_dx, y1 = (j+1)*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawWallGrid2D.threshold = 0.005;

	// g3d_extension gvextor field  created 2023.06.17, last updated 2023.06.17
	// g3d.drawWallVectorField2D(ctx, rotAngle, wallFunc, zFunc, vFunc, colorMode, inc [, showBox] )
	g3d.drawWallVectorField2D = function(ctx, rotAngle, wallFunc, zFunc, vFunc, colorMode, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, threshold = g3d.drawWallVectorField2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii; if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj+=inc) {
				let j=jj; if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;

				if ( wallFunc(i,nny-j-1) ) {
					const  x = i*dx, y = j*dx, z = 0, z1 = 2*g3d.cz0;
					g3d.drawRotatedLine(ctx, x,y,z,x,y,z1,'#888888',sc,xp,yp);
					continue;
				}

				const f = zFunc(i,nny-j-1);
				const x = i*dx, y = j*dx, z = f + g3d.cz0;
				let vx, vy;
				[ vx, vy ] = vFunc(i,nny-j-1);
				if ( vx*vx+vy*vy < threshold ) continue;
				const x1 = x + vx, y1 = y -vy, z1 = z;
				let colr;
				if ( colorMode==0 ) {
					const hue = Math.floor((Math.atan2(vy,vx))*180.0/Math.PI+180.0);
					colr = `hsl(${hue},100%,50%)`;
				} else if ( colorMode==1 ) {
					colr = (vx>0) ? "#3333ff" : "#ff3333";
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawWallVectorField2D.threshold = 0.5;


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {  // select theme
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setRevL() {
		const sr = 0 + document.getElementById("slct_RevL").selectedIndex;
		v_RevL = 50.0*Math.pow(2,sr);
		resetFlag = true;
	}

	function setDispMode() {  // select dispMode
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setSpeed() {
		v_nCalc = 1 + document.getElementById("slct_speed").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle(30,-15);
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, status, themeStr, meanRho ];
  }

  function pygetFieldData() {
    getFieldFlag = false;
    return [ cellKindArray, fieldArray, vectorArray ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setRevL:		setRevL,		// setRevL()
		setDispMode:	setDispMode,	// setDispMode()
		setSpeed:		setSpeed,		// setSpeed()
		viewHome:		viewHome,		// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, status, themeStr, meanRho ]
    pygetFieldData: pygetFieldData, // pygetFieldData() : return [ cellKindArray, fieldArray, vectorArray ]
	};

})(); // ====================  js076 module end  ====================


const js = js076;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js076]  air flows - lattice Boltzmann Method - 2D9V model</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="500" HEIGHT="480"></canvas>
<br>

<label>theme:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option selected>gas expansion</option><option>gas expansion+object</option><option>vortex</option>
<option>Couette flow</option><option>cavity flow</option><option>wind tunnel(180x90)</option>
<option selected>wind tunnel(120x60)</option>
</select>
    <span style="margin-right: 130px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>density(x,y) + flow</option><option>density(x,y) + powder</option>
<option>volticity(x,y) + powder</option><option>(speed(x,y)-vx0) + powder</option>
<option>grid density(x,y)</option><option>grid color:vorticity(x,y) z:density(x,y)</option>
<option>flow(x,y) z:density(x,y)</option>
<option selected>grid density(x,y) + powder</option><option>grid vorticity(x,y) + powder</option>
</select>
    <span style="margin-right: 20px;"></span>
<label>nCalc/frame:</label>
<select id="slct_speed" onChange="js.setSpeed()">
<option>1</option><option>2</option><option>3</option><option>4</option>
<option>5</option><option selected>6</option><option>7</option><option>8</option>
<option>9</option><option>10</option>
</select>
<br>

<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# exec html-js code, and python control

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ sysTime, status, themeStr, meanRho ] = eval_js( 'js.pygetData({})'.format(i) )
  print( f'i = {i:>2d},  time = {sysTime:>8.1f},  system status: {status}' )
  time.sleep(3)

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / change dispMode

import time

themeList = [
  '0: gas expansion', '1: gas expansion + object', '2: vortex', '3: Couette flow',
  '4: cavity flow', '5: wind tunnel(180x90)', '6: wind tunnel(120x60)' ]

dispModeList = [
  '0: density(x,y) + flow', '1: density(x,y) + powder', '2: volticity(x,y) + powder', '3: (speed(x,y)-vx0) + powder',
  '4: grid density(x,y)', '5: grid color:vorticity(x,y) z:density(x,y)', '6: flow(x,y) z:density(x,y)',
  '7: grid density(x,y) + powder', '8: grid vorticity(x,y) + powder' ]

# exec html-js code
exec_html_js()
print("-- start --")

# change dispMode and print
for theme in [ 0, 2, 4, 6 ]:
  eval_js( 'js.pysetTheme({})'.format(theme) )
  print("-- theme :", themeList[theme], " --")
  for dispMode in [ 1, 6, 7 ]:
    eval_js( 'js.pysetDispMode({})'.format(dispMode) )
    print("-- dispMode:", dispModeList[dispMode], " --")
    [ sysTime, status, themeStr, meanRho ] = eval_js( 'js.pygetData({})'.format(i) )
    print( f'\t time = {sysTime:>8.1f},  system status: {status}' )
    time.sleep(3)

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / get field and vector array and plot

import time
import numpy as np
import matplotlib.pyplot as plt

themeList = [
  '0: gas expansion', '1: gas expansion + object', '2: vortex', '3: Couette flow',
  '4: cavity flow', '5: wind tunnel(180x90)', '6: wind tunnel(120x60)' ]

# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(1)

# set theme
theme = 6 # '6: wind tunnel(120x60)'
eval_js( 'js.pysetTheme({})'.format(theme) )
print("-- set theme :", themeList[theme], " --")
time.sleep(5)

# get field and vector data
print("-- get pressure data array --")
[ cellKindArray, fieldArray, vectorArray ] = eval_js('js.pygetFieldData()')
print("field rows length:", len(fieldArray))
print("field columns length:", len(fieldArray[0]))

print("vector rows length:", len(vectorArray))
print("vector columns length:", len(vectorArray[0]))
print("vector length:", len(vectorArray[0][0]))

time.sleep(1)
# animation break to END
eval_js('js.breakLoop()')
print("-- end --")

# pressure field / image plot
Z = np.array(fieldArray).T
im = plt.imshow(Z, origin='lower', cmap='jet' )
plt.colorbar(im)
plt.title("density map")
plt.show()


# density field / surface plot with adjusted aspect ratio
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(projection='3d')

# Plot the surface
nx = len(fieldArray)
ny = len(fieldArray[0])
x = np.arange(0,nx)
y = np.arange(0,ny)
X, Y = np.meshgrid(x, y)
Z = np.array(fieldArray).T
ax.plot_surface(X,Y,Z, cmap='jet')

# Adjust aspect ratio
ax.set_box_aspect([1, 0.5, 0.5])  # [x, y, z] aspect ratios

plt.xlabel("x")
plt.ylabel("y")
plt.show()

# vector field plot
fig = plt.figure(figsize=(10, 5))
ax1 = fig.add_subplot(111)
vectorArray = np.array(vectorArray)
x = np.arange(0, len(vectorArray[0]))
y = np.arange(0, len(vectorArray))
X, Y = np.meshgrid(x, y)
U = vectorArray[:, :, 0]
V = vectorArray[:, :, 1]

scale_factor = 10
ax1.quiver(Y, X, U, V, scale=scale_factor )
plt.title("flow vector (vx,vy)")
plt.show()


In [ ]:
# use plotly Lib / https://plotly.com/python/3d-surface-plots/#multiple-3d-surface-plots

import plotly.graph_objects as go
import numpy as np

fig = go.Figure(go.Surface(z=np.array(fieldArray).T))

fig.update_layout(
        scene = {
            "xaxis": {"nticks": 20},
            "zaxis": {"nticks": 4},
            'camera_eye': {"x": 0, "y": -1, "z": 0.5},
            "aspectratio": {"x": 1.2, "y": 0.6, "z": 0.5}
        })

fig.show()